In [1]:
import multiprocessing as mp

# Hoeveel processen te starten?
Een goede maat is maximaal zo veel processen te starten als je CPU's hebt
* _Tenzij_ je vooral met Input/Output bezig bent (helemaal als het om netwerkverkeer gaat)
* He precieze gedrag is ook afhankelijk van de processorarchitectuur; experimenteren loont
* Een realistische harde bovengrens voor berekeningen is 2 x aantal cores
Je kunt makkelijk het aantal CPU's bepalen in multiprocessing

In [2]:
cpus = mp.cpu_count()

# Een gemakkelijke methode om ff wat taken te verdelen
## De multiprocessing.Pool class
* Zet een buffer aan werker-processen op, en vervolgens kun je met de functionele constructen apply(), map(), starmap() etc data automatisch verdelen 
* Dit lijkt al behoorlijk op een gebruikersvriendelijke library zoals Spark of Dask!
* Wij gaan in eerste instantie dit niet toepassen om beter de achterliggende processen te leren.
* Geeft ook minder controle over de communicatie en datastructuren.

In [3]:
def echo(x):
    print(x, "gekregen!")
    return x

def prime(x):
    #print("working on:",x)
    if x <= 0:
        print("Primality is undefined for 0 or less")
        return False
    if x == 1:
        print("No, 1 is not prime!")
        return False
    if x == 2:
        print("Yes, 2 is prime!")
        return True
    for i in range(2, x-1):                 # Dit kan ook (x // 2)+1
        if x % i == 0:
            #print(x, "is not prime!")
            return False
        
    #print(x, "is prime!")
    return True

if __name__ == "__main__":
    cpus = mp.cpu_count()
    with mp.Pool(cpus) as pool:
        results = pool.map(prime, range(1,49))

    print(results)
    

No, 1 is not prime!
Yes, 2 is prime!
[False, True, True, False, True, False, True, False, False, False, True, False, True, False, False, False, True, False, True, False, False, False, True, False, False, False, False, False, True, False, True, False, False, False, False, False, True, False, False, False, True, False, True, False, False, False, True, False]


# Communicatie mbv pipes
## Een subclass van queue die alleen toegangkelijk is voor 2 processen
* Soms wil je zeker weten dat data van het een gespecificeerd proces naar een ander gaat
* Queue's zijn een free-for-all; iedereen met de referentie mag er in kijken!
* "Pipes" zijn bidirectionele communicatiekanalen tussen twee processen
* De "semantiek" (m.a.w. de "taal" die je spreekt om het te bedienen) is hetzelfde als queue:
    * put() om data erin te stoppen (of put_nowait() )
    * get() om data erin te stoppen (of get_nowait() )

In [4]:
    from time import sleep
    from random import randint

    def todo(p, pipe):
        sleeptime = randint(1,5)
        sleep(sleeptime)
        outputQ.put("hoi! van process %s heeft %s sec geslapen" % (p , sleeptime))

    processes = {}
    outputQ = mp.Queue()

    for p in range(5):
        pipeP = mp.Pipe()
        temP = mp.Process(target=todo, args=(p,outputQ))
        processes[p] = {'process' : temP, 'pipe' : pipeP}
        temP.start()

    for p in processes:
        processes[p]['process'].join()

    for p in processes:
        pipeP = processes[p]['pipe']
        print(pipeP[1].get())

AttributeError: 'Connection' object has no attribute 'get'

In [ ]:
fp = open("bla", "r")
fp.readlines()

fp.read(8)
fp.seek(byte_offset)
